<a href="https://colab.research.google.com/github/Krahjotdaan/MachineLearning/blob/main/LinearModels_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ЛИНЕЙНЫЕ МОДЕЛИ ДЛЯ ЗАДАЧ РЕГРЕССИИ

# Самостоятельная практика по линейным моделям для задачи регрессии

Мы поработаем с данными о сообществах в США. Описание датасета:

http://archive.ics.uci.edu/ml/datasets/communities+and+crime

Датасет **crimedata** (в формате .csv)

Будем предсказывать количество насильственных преступлений относительно численности населения.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score

In [ ]:
data = pd.read_csv('crimedata.csv', na_values=["?"])

# оставим только необходимые признаки
requiredColumns = [5, 6] + list(range(11,26)) + list(range(32, 103)) + [145]
data = data[data.columns[requiredColumns]]

# некоторые значения целевой переменной пропущены,
# возьмем только те объекты, в которых y существует
X = data.loc[data['ViolentCrimesPerPop'].notnull(), :].drop('ViolentCrimesPerPop', axis=1)
y = data['ViolentCrimesPerPop'][X.index]

X.head()

In [ ]:
# Разобъем выборку на train и test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

### Задание 1. Простейшая регрессия.

* Обучите стаднартную линейную регрессию на тренировочной выборке (sklearn.linear_model.LinearRegression)

* Предскажите значения для X_train, X_test. Посчитайте метрики r2_score и MSE для них.

In [ ]:
lr = LinearRegression().fit(X_train, y_train)

y_pred_train = lr.predict(X_train)
y_pred_test = lr.predict(X_test)

print('Train: ', mean_squared_error(y_pred_train, y_train) ** 0.5, r2_score(y_pred_train, y_train))
print('Test: ', mean_squared_error(y_pred_test, y_test) ** 0.5, r2_score(y_pred_test, y_test))

Train:  349.69284765904166 0.5249004102969663
Test:  408.0747344515245 0.4628732137470942


### Задание 2. Регуляризация.

* Обучите Ridge-регрессию (sklearn.linear_model.Ridge) на обучающей выборке с alpha = 5.

* Предскажите значения и посчитайте метрики r2_score и mse.

* Изменилось ли качество ? Как ?

In [ ]:
lr = Ridge(5).fit(X_train, y_train)

y_pred_train = lr.predict(X_train)
y_pred_test = lr.predict(X_test)

print('Train: ', mean_squared_error(y_pred_train, y_train) ** 0.5, r2_score(y_pred_train, y_train))
print('Test: ', mean_squared_error(y_pred_test, y_test) ** 0.5, r2_score(y_pred_test, y_test))

Train:  350.4190665274027 0.5207213160830841
Test:  407.6491097985911 0.4627593185986346


### Задание 3. Масштабирование.

* Отмасштабируйте выборку с помощью sklearn.
* Проверьте, изменилось ли качество Ridge и обычной регрессии после масштабирования.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler()
X_train_scaled = pd.DataFrame(data=sc.fit_transform(X_train), columns=X_train.columns)
X_test_scaled = pd.DataFrame(data=sc.transform(X_test), columns=X_test.columns)

In [ ]:
lr = LinearRegression().fit(X_train_scaled, y_train)

y_pred_train = lr.predict(X_train_scaled)
y_pred_test = lr.predict(X_test_scaled)

print('Train: ', mean_squared_error(y_pred_train, y_train) ** 0.5, r2_score(y_pred_train, y_train))
print('Test: ', mean_squared_error(y_pred_test, y_test) ** 0.5, r2_score(y_pred_test, y_test))

Train:  349.7586740205084 0.5248972576977219
Test:  407.27673402025226 0.46472253185087786


In [ ]:
lr = Ridge(5).fit(X_train_scaled, y_train)

y_pred_train = lr.predict(X_train_scaled)
y_pred_test = lr.predict(X_test_scaled)

print('Train: ', mean_squared_error(y_pred_train, y_train) ** 0.5, r2_score(y_pred_train, y_train))
print('Test: ', mean_squared_error(y_pred_test, y_test) ** 0.5, r2_score(y_pred_test, y_test))

Train:  367.0427492804912 0.4005168559985931
Test:  389.05827199084376 0.3901709329600833


### Задание 4. Выбираем features.

* Проверьте, нужны ли нам признаки у которых маленькая дисперсия ?
Для этого оцените качество, если убрать из выборки признаки с дисперсией меньше 0.01

Для этого есть замечательный класс sklearn.feature_selection.VarianceThreshold.

In [ ]:
from sklearn.feature_selection import VarianceThreshold

# убираем все признаки, у которых дисперсия меньше < 0.01

vs_transformer = VarianceThreshold(0.01)

X_train_var = pd.DataFrame(data=vs_transformer.fit_transform(X_train_scaled),
                           columns=X_train_scaled.columns[vs_transformer.get_support()])
X_test_var = pd.DataFrame(data=vs_transformer.transform(X_test_scaled),
                          columns=X_test_scaled.columns[vs_transformer.get_support()])

In [ ]:
lr = LinearRegression().fit(X_train_var, y_train)

y_pred_train = lr.predict(X_train_var)
y_pred_test = lr.predict(X_test_var)

print('Train: ', mean_squared_error(y_pred_train, y_train) ** 0.5, r2_score(y_pred_train, y_train))
print('Test: ', mean_squared_error(y_pred_test, y_test) ** 0.5, r2_score(y_pred_test, y_test))

Train:  357.4825189768171 0.49263965740226745
Test:  375.6775183410547 0.42849471676558915


In [ ]:
lr = Ridge(5).fit(X_train_var, y_train)

y_pred_train = lr.predict(X_train_var)
y_pred_test = lr.predict(X_test_var)

print('Train: ', mean_squared_error(y_pred_train, y_train) ** 0.5, r2_score(y_pred_train, y_train))
print('Test: ', mean_squared_error(y_pred_test, y_test) ** 0.5, r2_score(y_pred_test, y_test))

Train:  373.04396655112294 0.3696871561573697
Test:  378.63050886921644 0.32612141442279363


### Задание 5. Lasso как способ нахождения хороших признаков.

* Давайте обучим Lasso и найдем самые значимые признаки с точки зрения данной модели регрессии.

* Покажите, какие признаки ушли, попробуйте разные alpha, чтобы убрать больше коофициентов.

* Сравните качество!

In [ ]:
lr = Lasso(0.0001).fit(X_train, y_train)

y_pred_train = lr.predict(X_train)
y_pred_test = lr.predict(X_test)

print('Train: ', mean_squared_error(y_pred_train, y_train) ** 0.5, r2_score(y_pred_train, y_train))
print('Test: ', mean_squared_error(y_pred_test, y_test) ** 0.5, r2_score(y_pred_test, y_test))

Train:  349.953568641173 0.5234860320076624
Test:  408.0742677644148 0.46262366515586073


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.175e+07, tolerance: 5.069e+04
  model = cd_fast.enet_coordinate_descent(


In [ ]:
list(zip(X.columns, lr.coef_))

[('population', 0.0001881568116846154),
 ('householdsize', -110.6271215173407),
 ('agePct12t21', 15.468732376517188),
 ('agePct12t29', -33.49312158945201),
 ('agePct16t24', -1.7910769803845674),
 ('agePct65up', -15.874458571069953),
 ('numbUrban', -0.0014701026009623479),
 ('pctUrban', 1.3470342365158905),
 ('medIncome', -0.012022918293698063),
 ('pctWWage', -6.79455694870038),
 ('pctWFarmSelf', 22.262853120892405),
 ('pctWInvInc', -5.13709535624599),
 ('pctWSocSec', 9.759380112387143),
 ('pctWPubAsst', -2.9918859682949033),
 ('pctWRetire', -7.403574125521583),
 ('medFamInc', 0.009001317154819253),
 ('perCapInc', 0.0033769631204070916),
 ('NumUnderPov', 0.007864284754829247),
 ('PctPopUnderPov', -3.8644535564639657),
 ('PctLess9thGrade', -16.28267674876414),
 ('PctNotHSGrad', 9.050920061787528),
 ('PctBSorMore', -2.2134369292542213),
 ('PctUnemployed', 1.0334004645327313),
 ('PctEmploy', 8.634477230802958),
 ('PctEmplManu', -3.9233238262625068),
 ('PctEmplProfServ', -2.333163728778412)